In [51]:
import yfinance as yf

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from  datetime import date, timedelta

In [52]:
date_now = str(date.today() - timedelta(days=1))
df_apple_to_csv = yf.download("AAPL", start="2020-01-01", end=date_now)
df_microsoft_to_csv = yf.download("MSFT", start="2020-01-01", end=date_now)
df_amazon_to_csv = yf.download("AMZN", start="2020-01-01", end=date_now)
df_nvidia_to_csv = yf.download("NVDA", start="2020-01-01", end=date_now)


apple_path = 'datasets/apple.csv'
microsoft_path = 'datasets/microsoft.csv'
amazon_path = 'datasets/amazon.csv'
nvidia_path = 'datasets/nvidia.csv'

df_apple_to_csv.to_csv(apple_path)
df_microsoft_to_csv.to_csv(microsoft_path)
df_amazon_to_csv.to_csv(amazon_path)
df_nvidia_to_csv.to_csv(nvidia_path)

df_apple = pd.read_csv(apple_path)
df_microsoft = pd.read_csv(microsoft_path)
df_amazon = pd.read_csv(amazon_path)
df_nvidia = pd.read_csv(nvidia_path)

C:\Users\acer\AppData\Local\Temp\ipykernel_20156\2667087305.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_apple_to_csv = yf.download("AAPL", start="2020-01-01", end=date_now)
[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_20156\2667087305.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_microsoft_to_csv = yf.download("MSFT", start="2020-01-01", end=date_now)
[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_20156\2667087305.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_amazon_to_csv = yf.download("AMZN", start="2020-01-01", end=date_now)
[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_20156\2667087305.py:5: FutureWarning: YF.download() has changed argument auto_adjust defaul

In [53]:
df_apple['Date'] = df_apple['Price']
df_apple = df_apple.drop(index=[0,1], columns='Price')

df_microsoft['Date'] = df_microsoft['Price']
df_microsoft = df_microsoft.drop(index=[0,1], columns='Price')

df_amazon['Date'] = df_amazon['Price']
df_amazon = df_amazon.drop(index=[0,1], columns='Price')

df_nvidia['Date'] = df_nvidia['Price']
df_nvidia = df_nvidia.drop(index=[0,1], columns='Price')

In [54]:
#INITIAL
dfs = [df_apple, df_amazon, df_microsoft, df_nvidia]
window = 14
short_period = 12
long_period = 26
signal_line_period = 9
stocks = ['AAPL', 'AMZN', 'MSFT', 'NVDA']

In [55]:
#CONVERTING TO CORRECT FEATURE TYPES
def convert_type(df):
    df['Close'] = df['Close'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Open'] = df['Open'].astype(float)
    df['Volume'] = df['Volume'].astype(int)
    df['Date'] = pd.to_datetime(df['Date'])

In [56]:
#FEATURE ENGINEERING
def engineer_features(df):
    df['Price Change'] = df['Close'].diff().fillna(0)

    df['Gain'] = df.loc[df['Price Change'] > 0, 'Price Change']
    df['Loss'] = -df.loc[df['Price Change'] < 0, 'Price Change']

    df['Gain'] = df['Gain'].fillna(0)
    df['Loss'] = df['Loss'].fillna(0)

    df['Average Gain'] = df['Gain'].rolling(window=window).mean().fillna(0)
    df['Average Loss'] = df['Loss'].rolling(window=window).mean().fillna(0)

    df['RS'] = df['Average Gain'] / df['Average Loss']

    df['RSI'] = 100 - (100 / (1 + df['RS']))

    df['Short EMA'] = df['Close'].ewm(span=short_period,adjust=False).mean()
    df['Long EMA'] = df['Close'].ewm(span=long_period, adjust=False).mean()

    df['MACD'] = df['Short EMA'] - df['Long EMA']

    df['Signal Line'] = df['MACD'].ewm(span=signal_line_period, adjust=False).mean()
    
    df['MACD Histogram'] = df['MACD'] - df['Signal Line']
    
    df['Year'] = df['Date'].dt.year
    df['Week'] = df['Date'].dt.weekday
    
    df['Year-Month'] = df['Date'].dt.to_period('M').astype(str)
    #df['Week']
    
    return df

In [57]:
#DROPPING NULL AND UNIMPORTANT COLUMNS/ROWS
def drop_features_and_na(df):
    df = df.drop(columns=['Gain', 'Loss'])
    df = df.dropna()
    return df

In [58]:
#PRINTING SEABORN PLOTS
def print_dmy_plots(df, stock):
    #1W
    # plt.figure(figsize=(15,7))
    # sns.lineplot(data=df.loc[len(df) - 7::, ['Date','Close']], x='Date', y='Close', errorbar=None)
    # plt.title(f'1 Week for {stock}')
    # # plt.show()
    
    # #1M
    # plt.figure(figsize=(15,7))
    # sns.lineplot(data=df.loc[len(df) - 31::, ['Date','Close']], x='Date', y='Close', errorbar=None)
    # plt.title(f'1 Month for {stock}')
    # # plt.show()
    
    # #3M
    # plt.figure(figsize=(15,7))
    # sns.lineplot(data=df.loc[len(df) - (31 * 3)::, ['Date','Close']], x='Date', y='Close', errorbar=None)
    # plt.title(f'3 Months for {stock}')
    # # plt.show()
    
    # #1Y 
    # plt.figure(figsize=(15,7))
    # sns.lineplot(data=df.loc[len(df) - 365::, ['Date','Close']], x='Date', y='Close', errorbar=None)
    # plt.title(f'1 Year for {stock}')
    # # plt.show()
    
    # #5Y 
    # plt.figure(figsize=(15,7))
    # sns.lineplot(data=df.loc[len(df) - (365 * 5)::, ['Date','Close']], x='Date', y='Close', errorbar=None)
    # plt.title(f'5 Years for {stock}')
    # # plt.show()
    
    # #ALL 
    # plt.figure(figsize=(15,7))
    # sns.lineplot(data=df, x='Date', y='Close', errorbar=None)
    # plt.title(f'All time for {stock}')
    # plt.show()
    pass

In [59]:
#1D INTERVAL
def print_1d_plot(df, stock):
    fig = px.line(data_frame=df, x='Date', y='Close')
    fig.update_layout(width=1920, height=1080)
    fig.write_html(f"../../frontend/public/{stock}1D.html")
    print(f'1 Day Interval trend for {stock}, downloaded!')

In [60]:

#TODO: 1W INTERVAL
def print_1w_plot(df, stock):
    fig = px.line(data_frame=df, x='Date', y='Close')
    fig.update_layout(width=1920, height=1080)
    fig.write_html(f"../../frontend/public/{stock}1W.html")
    print(f'1 Week Interval trend for {stock}, downloaded!')

In [61]:
#1M INTERVAL
def print_1m_plot(df, stock):
    fig = px.line(data_frame=df, x='Year-Month', y='Close')
    fig.update_layout(width=1920, height=1080)
    fig.write_html(f"../../frontend/public/{stock}1M.html")
    print(f'1 Month Interval trend for {stock}, downloaded!')

In [62]:
#1Y INTERVAL
def print_1y_plot(df, stock):
    fig = px.line(data_frame=df, x='Year', y='Close')
    fig.update_layout(width=1920, height=1080)
    fig.write_html(f"../../frontend/public/{stock}1Y.html")
    print(f'1 Year Interval trend for {stock}, downloaded!')

In [63]:
def print_plots(df, stock):
    print_1d_plot(df, stock)
    # print_1w_plot(df, stock)
    print_1m_plot(df, stock)
    print_1y_plot(df, stock)
    pass

In [64]:
for i,df in enumerate(dfs):
    convert_type(df)
    df = engineer_features(df)
    df = drop_features_and_na(df)
    
    # print('---------------------------------------------------------------------------')
    print(f"{stocks[i]} is good to go!")
    # print(df.head())
    
    print_dmy_plots(df, stocks[i])
    # print_plots(df, stocks[i])

AAPL is good to go!
AMZN is good to go!
MSFT is good to go!
NVDA is good to go!
